In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
#df = pd.read_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240616/full_step1_6.csv')

df = pd.read_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20241002/full_step1_6.csv')
#df = df[:1000]

In [3]:
"""
'Response'轉換 => {'Withdraws':1, 'No Response':2, 'Others':3}
"""
conditions = [
    (df['Response'] == 'Withdraws'),
    (df['Response'] == 'No Response')
]
choices = [1, 2]
df['Response'] = np.select(conditions, choices, default=3)
df[df['Response'].notna()]['Response']

0      3
1      3
2      3
3      3
4      3
      ..
719    3
720    3
721    3
722    3
723    3
Name: Response, Length: 724, dtype: int32

In [4]:
for i in df.columns:
    print(i)

stay_id
use_vent
date
SaO2
Respiration
Heart Rate
Systemic Systolic
Systemic Diastolic
Systemic Mean
Compliance
FiO2
Minute Ventilation
Mean Airway Pressure
Peak Airway Pressure
PEEP
PC mode
Pressure Support
Plateau
Tidal Volume
Respiratory Rate
ROXindex
RASS
Platelets x1000
WBC x1000
Hgb
Albumin
Total Protein
Total Bilirubin
PO2
PaCO2
Glucose
BUN
pH
Sodium
Potassium
Magnesium
Calcium
Chloride
creatinine
HCO3
Phosphate
Alkaline Phos.
AST (SGOT)
ALT (SGPT)
PT-INR
Vasopressor
Relaxant
Sedation
PPI
Pain control
total
Fluid_intake_value
Nutrition_Enteral_value
Urine_value
Aspergillus
Candida
Abdomen
Blood
Respiratory tract
Skin and soft tissue
Urinary tract
Others
dod
dod_3day
dod_7day
dod_30day
dod_60day
dod_90day
InvasiveVent
tracheostomy
NonInvasiveVent
SupplementalOxygen
HFNC
Anxiety
Assess for anxiety, depression, or delirium which may contribut
Asystole
Blood Transfusion
Consult to pastoral care, social services, palliative care, psy
DNAR (Do Not Attempt Resuscitation) [DNR]
DNAR (Do

In [5]:
#整併DNR => 併為'DNR_Old'
columns_to_drop = ['DNAR (Do Not Attempt Resuscitation) [DNR]',
                   'DNAR (Do Not Attempt Resuscitation) [DNR] / DNI',
                   'DNI (do not intubate)',
                   'DNR (do not resuscitate)',
                   'DNR / DNI',
                   'dnr']
DNR_values = df[columns_to_drop].max(axis=1)
df.drop(columns=columns_to_drop, inplace=True)
df['DNR_old'] = DNR_values
df['DNR_old'] = df['DNR_old'].fillna(0)

In [6]:
#CPOT相關特徵補0
columns_CPOT = ['Muscle','Vocalization','Body Movements','Facial Expressions','CPOT(SUM)','CPOT(SUM)']

for col in columns_CPOT:
    df[col] = df[col].fillna(0)



In [7]:
#衍生特徵移動到動態特徵索引
cols_to_move = ['Consecutive_Vasopressor_Over3', 'Consecutive_Vasopressor_Over7', 'Consecutive_avg_FiO2_Over50', 'Consecutive_avg_FiO2_Over60','DNR_old']
cols_to_move = cols_to_move + ['Full_code_interval', 'Strength Arm', 'Strength Leg']
#cols_to_move = cols_to_move + ['use_vent_acc', 'dialysis_acc', 'dialysis_since_last_event','use_vent_consecutive_7days']
#cols_to_move = cols_to_move + ['use_vent_acc', 'dialysis_acc', 'dialysis_since_last_event', 'use_vasopressor_acc']
cols_to_move = cols_to_move + ['dialysis_acc', 'dialysis_since_last_event']
cols_to_move = cols_to_move + ['dialysis_over3', 'dialysis_over7']


for col in cols_to_move[::-1]:
    df.insert(10, col, df.pop(col))

In [8]:
for i in df.columns:
    print(i)

stay_id
use_vent
date
SaO2
Respiration
Heart Rate
Systemic Systolic
Systemic Diastolic
Systemic Mean
Compliance
Consecutive_Vasopressor_Over3
Consecutive_Vasopressor_Over7
Consecutive_avg_FiO2_Over50
Consecutive_avg_FiO2_Over60
DNR_old
Full_code_interval
Strength Arm
Strength Leg
dialysis_acc
dialysis_since_last_event
dialysis_over3
dialysis_over7
FiO2
Minute Ventilation
Mean Airway Pressure
Peak Airway Pressure
PEEP
PC mode
Pressure Support
Plateau
Tidal Volume
Respiratory Rate
ROXindex
RASS
Platelets x1000
WBC x1000
Hgb
Albumin
Total Protein
Total Bilirubin
PO2
PaCO2
Glucose
BUN
pH
Sodium
Potassium
Magnesium
Calcium
Chloride
creatinine
HCO3
Phosphate
Alkaline Phos.
AST (SGOT)
ALT (SGPT)
PT-INR
Vasopressor
Relaxant
Sedation
PPI
Pain control
total
Fluid_intake_value
Nutrition_Enteral_value
Urine_value
Aspergillus
Candida
Abdomen
Blood
Respiratory tract
Skin and soft tissue
Urinary tract
Others
dod
dod_3day
dod_7day
dod_30day
dod_60day
dod_90day
InvasiveVent
tracheostomy
NonInvasiveVent

In [9]:
"""
當天為1，前後兩天設置為1
"""
def impute_Gram_stain(df_in):
    Gram_stain_feature = ['Abdomen','Blood','Respiratory tract','Skin and soft tissue','Urinary tract','Others']
    df_out = df_in.copy()
    for col in Gram_stain_feature:
        df_out[col] = df_out[col].rolling(window=5, center=True, min_periods=1).apply(lambda x: (x.sum() >= 1) * 1, raw=False)
    return df_out


"""
棄用
"""
def impute_PEEP_FiO2(df_P,avg_PEEP,avg_FiO2):
    
    #沒使用呼吸器的預設值
    PEEP = 5
    FiO2 = 30
    df_P.loc[df_P['use_vent'] == 0, ['PEEP']] = PEEP
    df_P.loc[df_P['use_vent'] == 0, ['FiO2']] = FiO2
    
    fill_interval = False
    for i in range(len(df_P)):
        if df_P.loc[i, 'use_vent'] == 1 and not fill_interval:
            fill_interval = True
            start_index = i
        elif df_P.loc[i, 'use_vent'] == 0 and fill_interval:
            fill_interval = False
            end_index = i
            subset = df_P.loc[start_index:end_index - 1].copy()
            subset['PEEP'] = subset['PEEP'].interpolate(method='linear', limit_direction='both')
            subset['FiO2'] = subset['FiO2'].interpolate(method='linear', limit_direction='both')
            subset['PEEP'].fillna(avg_PEEP, inplace=True)
            subset['FiO2'].fillna(avg_FiO2, inplace=True)
            df_P.loc[start_index:end_index - 1] = subset
    return df_P



"""
呼吸器特徵補值
"""
def impute_vent_feature(df_input,df_Mean):
    df_P = df_input.copy()

    vent_feature_list = ['PEEP','FiO2','Mean Airway Pressure','Peak Airway Pressure','RASS','SOFA-respiration']
    PEEP = 5
    FiO2 = 30
    df_P.loc[df_P['use_vent'] == 0, ['PEEP']] = PEEP
    df_P.loc[df_P['use_vent'] == 0, ['FiO2']] = FiO2
    
    fill_interval = False
    for i in range(len(df_P)):
        if df_P.loc[i, 'use_vent'] == 1 and not fill_interval:
            fill_interval = True
            start_index = i
        elif df_P.loc[i, 'use_vent'] == 0 and fill_interval:
            fill_interval = False
            end_index = i
            subset = df_P.loc[start_index:end_index - 1].copy()
            
            for feature_name in vent_feature_list:
                subset[feature_name].fillna(method='ffill', inplace=True)
                subset[feature_name].fillna(df_Mean, inplace=True)
            df_P.loc[start_index:end_index - 1] = subset
            
    for feature_name in vent_feature_list:
        df_P.loc[:, feature_name].fillna(df_Mean[feature_name], inplace=True)
    
    return df_P


def impute_strength_feature(df_input):
    df_P = df_input.copy()
    strength_feature_list = ['Strength Arm', 'Strength Leg']
    df_P[strength_feature_list] = df_P[strength_feature_list].fillna(method='ffill')        
    df_P[strength_feature_list] = df_P[strength_feature_list].fillna(0)     
    
    return df_P

"""
棄用
"""
def remove_outliers(series):
    if  series.dtype == object:
        print(f'object....{series.name}')
        return series
    if series.max() == 1 and series.min() == 0:
        print(f'found 0/1!....{series.name}')
        return series
    
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    return series[(series >= Q1 - 1.5 * IQR) & (series <= Q3 + 1.5 * IQR)]

"""
計算是否使用呼吸器連續超過3天
"""
def set_use_vent_over3(df_P):
    df_P.insert(10, 'use_vent_over3', 0)
    df_P['rolling_sum'] = df_P['use_vent'].rolling(window=3, min_periods=1).sum()
    df_P.loc[df_P.index[2:], 'use_vent_over3'] = df_P['rolling_sum'].gt(2).astype(int)
    df_P.drop(columns=['rolling_sum'], inplace=True)
    return df_P

In [10]:
#移除缺失率過高的特徵
#drop_cols = ['icu_intime','icu_outtime','PC mode','Total Protein','Compliance','Albumin','Alkaline Phos.','Total Bilirubin','ALT (SGPT)','AST (SGOT)','Respiration','Pressure Support','ROXindex','Troponin - I','PTT_Ratio']
drop_cols = ['Relaxant','PC mode','Total Protein','Compliance','Albumin','Alkaline Phos.','Total Bilirubin','ALT (SGPT)','AST (SGOT)','Respiration','Pressure Support','ROXindex','Troponin - I','PTT_Ratio']
df = df.drop(drop_cols,axis = 1)

missing_data_ratios = df.isna().mean()
cols_with_missing_data = missing_data_ratios[missing_data_ratios == 1].index
df = df.drop(cols_with_missing_data, axis=1)

In [11]:
""" Label Embedding """
object_col_list = []
le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    if col in ['Strength L Arm', 'Strength L Leg', 'Strength R Arm', 'Strength R Leg']:
        df[col].fillna(-1, inplace=True) 
    if col == 'date' or col == 'data_type' or col == 'ventilation_status': 
        continue
    df[col].fillna("NULL", inplace=True) 
    df[col] = le.fit_transform(df[col])
    object_col_list.append(col)
    
print(object_col_list)

""" Fill missing values with the binary column"""
binary_cols = df.columns[(df.isin([0, 1, np.nan])).all()]
df[binary_cols] = df[binary_cols].fillna(0)
print(binary_cols)


#df[['Strength L Arm', 'Strength L Leg', 'Strength R Arm', 'Strength R Leg']]  = df[['Strength L Arm', 'Strength L Leg', 'Strength R Arm', 'Strength R Leg']].fillna(-1)
#exclude_cols = ['date', 'data_type', 'ventilation_status','FiO2','PEEP']
#dont_remove_outlier_cols = exclude_cols + list(binary_cols) + object_col_list
#df = df.apply(lambda col: remove_outliers(col) if col.name not in dont_remove_outlier_cols else col, axis=0)

['Coping/Knowledge Deficit  NCP - Expected outcomes', 'gender', 'race']
Index(['use_vent', 'Consecutive_Vasopressor_Over3',
       'Consecutive_Vasopressor_Over7', 'Consecutive_avg_FiO2_Over50',
       'Consecutive_avg_FiO2_Over60', 'DNR_old', 'dialysis_acc',
       'dialysis_over3', 'dialysis_over7', 'Vasopressor',
       ...
       'DM_acute', 'DM_Chronic', 'Hemiplegia', 'Renal', 'Malignancy', 'LD',
       'MST', 'AIDS', 'Spouse', 'use_vent_consecutive_7days'],
      dtype='object', length=112)


In [ ]:
import os

""" 
mean of column 
避免後續多次計算均值
"""
# 路徑
#file_path = f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240616/df_Mean.csv'

file_path = f'C:/Users/USER/M1326168/MIMIC/DNR/20241002/df_Mean.csv'

if os.path.exists(file_path):
    df_Mean = pd.read_csv(file_path)
else:
    df_Mean = pd.DataFrame(df.mean()).T
    df_Mean.to_csv(file_path, index=False) 

C:\Users\USER\AppData\Local\Temp\ipykernel_19256\837601145.py:15: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_Mean = pd.DataFrame(df.mean()).T


In [13]:
"""
avg_PEEP = df[df['use_vent'] == 1]['PEEP'].mean()
avg_FiO2 = df[df['use_vent'] == 1]['FiO2'].mean()
"""

"\navg_PEEP = df[df['use_vent'] == 1]['PEEP'].mean()\navg_FiO2 = df[df['use_vent'] == 1]['FiO2'].mean()\n"

In [14]:
df_result = pd.DataFrame()
df_result_list = []
mean = df_Mean.mean()

distinct_stay_id = df['stay_id'].unique()
for stay_ids in tqdm(distinct_stay_id):     
    df_P = df[df['stay_id'] == stay_ids]
    df_P.reset_index(drop=True, inplace=True)
    
    df_P = impute_strength_feature(df_P)
    df_P = impute_vent_feature(df_P,mean)
    df_P = impute_Gram_stain(df_P)
    df_P = set_use_vent_over3(df_P)
    
    df_P.interpolate(method='linear',limit_direction='both', inplace=True)
    df_P.interpolate(method='pad',limit_direction='forward', inplace=True)
    df_P.interpolate(method='bfill',limit_direction='backward', inplace=True)
    df_P.fillna(mean, inplace=True)
    
    df_result_list.append(df_P)
    

100%|██████████| 724/724 [00:48<00:00, 14.80it/s]


In [15]:
df_result = pd.concat(df_result_list, ignore_index=True)

In [16]:
"""
ICU類別合併
"""
df_result['ICU_Class_1'] = df_result[['CCU', 'MICU', 'MICU/SICU']].max(axis=1)
df_result.drop(['CCU', 'MICU', 'MICU/SICU'], axis=1, inplace=True)

df_result['ICU_Class_2'] = df_result[['Neuro Intermediate','Neuro Stepdown']].max(axis=1)
df_result.drop(['Neuro Intermediate','Neuro Stepdown'], axis=1, inplace=True)

df_result['ICU_Class_3'] = df_result[['TSICU']].max(axis=1)
df_result.drop(['TSICU'], axis=1, inplace=True)

cols_to_move = cols_to_move + ['dialysis_over3', 'dialysis_over7', 'ICU_Class_1', 'ICU_Class_2', 'ICU_Class_3']

for col in cols_to_move[::-1]:
    df_result.insert(10, col, df_result.pop(col))

In [ ]:
#df_result.to_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240616/full_step2.csv',index = False)

df_result.to_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20241002/full_step2.csv',index = False)

In [18]:
for i in (df_result.columns):
    print(f'{i}')

stay_id
use_vent
date
SaO2
Heart Rate
Systemic Systolic
Systemic Diastolic
Systemic Mean
Consecutive_avg_FiO2_Over50
Consecutive_Vasopressor_Over7
Consecutive_Vasopressor_Over3
Consecutive_avg_FiO2_Over60
DNR_old
Full_code_interval
Strength Arm
Strength Leg
dialysis_acc
dialysis_since_last_event
dialysis_over3
dialysis_over7
ICU_Class_1
ICU_Class_2
ICU_Class_3
use_vent_over3
FiO2
Minute Ventilation
Mean Airway Pressure
Peak Airway Pressure
PEEP
Plateau
Tidal Volume
Respiratory Rate
RASS
Platelets x1000
WBC x1000
Hgb
PO2
PaCO2
Glucose
BUN
pH
Sodium
Potassium
Magnesium
Calcium
Chloride
creatinine
HCO3
Phosphate
PT-INR
Vasopressor
Sedation
PPI
Pain control
total
Fluid_intake_value
Nutrition_Enteral_value
Urine_value
Aspergillus
Candida
Abdomen
Blood
Respiratory tract
Skin and soft tissue
Urinary tract
Others
dod
dod_3day
dod_7day
dod_30day
dod_60day
dod_90day
InvasiveVent
tracheostomy
NonInvasiveVent
SupplementalOxygen
HFNC
Anxiety
Assess for anxiety, depression, or delirium which may con